In [1]:
import requests, json

def match(conditions):
    return requests.post("http://localhost:5004/match_api", json=conditions).json()

def get_annotated_data():
    url = 'https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/models/match_pubmed_affiliations_with_countries_v3.json'
    data = requests.get(url).json()
    return data

def compute_precision_recall(match_type, index_prefix=''):

    data = get_annotated_data()

    nb_TP, nb_FP, nb_FN = 0, 0, 0
    false_positive, false_negative = [], []
    for ix, d in enumerate(data):
        if ix%100==0:
            print(ix, end=',')
        if d.get(match_type):
            res = match({'query': d['label'], 'year':'2020', 'type': match_type, 'index_prefix': index_prefix})
            for x in res['results']:
                if x in d[match_type]:
                    nb_TP += 1
                else:
                    nb_FP += 1
                    false_positive.append(d)
            for x in d[match_type]:
                if x not in res['results']:
                    nb_FN += 1
                    false_negative.append(d)

    precision = nb_TP / (nb_TP + nb_FP)
    recall = nb_TP / (nb_TP + nb_FN)
    res = {'precision' : precision, 'recall' : recall}
    return {'res': res, 'false_positive': false_positive, 'false_negative': false_negative} 


In [2]:
metrics_grid = compute_precision_recall(match_type = 'grid', index_prefix='matcher')

0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,4400,4500,4600,4700,

In [3]:
metrics_grid['res']

{'precision': 0.7443415637860082, 'recall': 0.5708086785009862}

In [4]:
len(metrics_grid['false_positive'])

497